<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/09_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第9章 深層学習

ディープニューラルネットワークでの識別をKerasでコーディングします。

## 畳み込みネットワークによる画像認識

## 準備

必要なライブラリ等を読み込みます。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import np_utils

Using TensorFlow backend.


## データの読み込み

第8章と同じMNISTデータ（学習用60000事例、評価用10000事例、それぞれ28x28の行列）を使います。

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

入力を3次元テンソルから4次元テンソルに変換します。各次元は、データ数、縦のピクセル数、横のピクセル数、チャネル数（グレースケース画像は1）を表します。

In [3]:

X_train = X_train.reshape(X_train.shape[0] , 28, 28, 1) 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1) 

値の型をTensorflowで演算に用いられるfloat32に変換します。

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

値のとりうる範囲を0-255から0-1に変換します。

In [5]:
X_train /= 255
X_test /= 255

畳み込みネットワークで学習します。

In [6]:
model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 56s 931us/step - loss: 0.2994 - accuracy: 0.9117
Epoch 2/5
60000/60000 [==============================] - 54s 905us/step - loss: 0.0799 - accuracy: 0.9757
Epoch 3/5
60000/60000 [==============================] - 54s 906us/step - loss: 0.0576 - accuracy: 0.9821
Epoch 4/5
60000/60000 [==============================] - 57s 949us/step - loss: 0.0448 - accuracy: 0.9864
Epoch 5/5
60000/60000 [==============================] - 50s 841us/step - loss: 0.0393 - accuracy: 0.9877


評価用データで評価します。

In [8]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 147us/step
Test accuracy: 0.9854999780654907


## リカレントニューラルネットワークによる自然言語処理

IMDBデータは映画のレビューに対して、P/N(肯定/否定)のラベルが付いた学習データです。学習用に25000事例、評価用に25000事例用意されていて、PNの割合はそれぞれ50%です。
各レビューは単語列ではなく、単語インデックスの系列として表現されています。

ここでは、頻度上位10000語を対象とし、データの大きさは先頭の50単語に限定します。

In [9]:
from keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional

In [10]:
max_features = 10000
maxlen = 50
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

単語インデックスを単語に戻して、元のデータを確認します。インデックスは"padding", "start of sequence","unknown"にそれぞれ0,1,2が割り当てられているので、3つずらして対応させます。

In [11]:
word_index = keras.datasets.imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

"grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

単純なRNNを構成して学習させます。

In [12]:
model = Sequential([
    Embedding(max_features, 128),
    SimpleRNN(64),
    Dense(1, activation='sigmoid')
]) 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                12352     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 1,292,417
Trainable params: 1,292,417
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 53s 3ms/step - loss: 0.5446 - acc: 0.7050 - val_loss: 0.4831 - val_acc: 0.7676
Epoch 2/2
20000/20000 [==============================] - 53s 3ms/step - loss: 0.3211 - acc: 0.8650 - val_loss: 0.4926 - val_acc: 0.7896


In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

25000/25000 [==============================] - 12s 498us/step
Test accuracy: 0.7932400107383728


## 課題1

畳み込みネットワークにおいて使用するデータをfasion MNISTに変更し、認識・評価を行ってください。また、余力があればdropoutの効果を検証してください。

参考コード
https://keras.io/examples/mnist_cnn/

## 課題2

リカレントニューラルネットワークにおいて、双方向LSTMへのネットワーク構造の変更やattentionの導入による性能の変化を確認してください。